# Sending data to Kafka using python

In [156]:
!pip install kafka
!pip install kafka-python
!pip install confluent-kafka

In [157]:
from kafka import KafkaProducer
import pandas as pd
import json
pd.set_option('display.max_columns', None)

# Data is been sorted in proper format

In [158]:
%%time
df = pd.read_json("1646893713217-service (output from script).json", convert_dates=False)
df

Wall time: 144 ms


,timestamp,type,name,ip,mac,latlon,ip4,ip6,lon,lat,deviceHash,incomingPkts,outgoingPkts,internalPkts,totalPkts,totalExternalPkts,incomingBytes,outgoingBytes,internalBytes,totalBytes,totalExternalBytes
0,1646893713217,Client,icmp,10.236.141.200,04b0e711fc18,NaN,183274952,NaN,NaN,NaN,NaN,0,25,0,25,25,0,2186,0,2186,2186
1,1646893713217,Server,icmp,194.48.132.18,None,NaN,-1037007854,NaN,NaN,NaN,NaN,25,0,0,25,25,2186,0,0,2186,2186
2,1646893713217,Client,unknown,10.110.19.88,fa163ea3e5a6,NaN,174986072,NaN,NaN,NaN,NaN,0,0,2006762,2006762,0,0,0,215466404,215466404,0
3,1646893713217,Client,unknown,10.115.128.2,04b0e711fc06,NaN,175341570,NaN,NaN,NaN,NaN,0,0,22,22,0,0,0,1484,1484,0
4,1646893713217,Client,unknown,10.115.128.1,04b0e711fc06,NaN,175341569,NaN,NaN,NaN,NaN,0,0,71,71,0,0,0,3879,3879,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13617,1646893713217,Server,unknown,10.131.211.1,None,NaN,176411393,NaN,NaN,NaN,NaN,0,0,387,387,0,0,0,43026,43026,0
13618,1646893713217,Server,unknown,10.131.199.162,None,NaN,176408482,NaN,NaN,NaN,NaN,0,0,418,418,0,0,0,43756,43756,0
13619,1646893713217,Server,unknown,10.131.170.201,None,NaN,176401097,NaN,NaN,NaN,NaN,0,0,255,255,0,0,0,24458,24458,0
13620,1646893713217,Server,unknown,10.131.58.73,None,NaN,176372297,NaN,NaN,NaN,NaN,0,0,250,250,0,0,0,23148,23148,0


In [64]:
# df['timestamp'].nunique()

In [159]:
# df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
# df['timestamp'] = df['timestamp'].dt.strftime("%Y-%m-%d-%H:%M:%S")
# df['timestamp']

In [160]:
# df.info()

### Sending 1 message as 1 list

In [161]:
str(df.iloc[0,:].values.tolist())  # 1 row = 1 message in a form of single list

"[1646893713217, 'Client', 'icmp', '10.236.141.200', '04b0e711fc18', nan, 183274952, nan, nan, nan, nan, 0, 25, 0, 25, 25, 0, 2186, 0, 2186, 2186]"

In [67]:
import confluent_kafka.admin, pprint

with open("config.json") as jfile:
        config_data = json.load(jfile)
    
topic_name =  config_data["topic_name"]
kafka_host = config_data["kafka_host"]

conf        = {'bootstrap.servers': kafka_host}
kafka_admin = confluent_kafka.admin.AdminClient(conf)

# new_topic   = confluent_kafka.admin.NewTopic('topic100', 1, 1) # Number-of-partitions  = 1
                                                                 # Number-of-replicas    = 1
# kafka_admin.create_topics([new_topic,]) # CREATE (a list(), so you can create multiple).

topic_list = kafka_admin.list_topics().topics # LIST OUT ALL THE TOPICS
print(topic_list.keys())

dict_keys(['test_topic_1', '__consumer_offsets'])


In [17]:
# kafka_admin.delete_topics(['test_topic_3',]) # DELETING A TOPIC

{'test_topic_3': <Future at 0x1ff8a015dc8 state=running>}

In [16]:
if  topic_name in list(topic_list.keys()):
    # print('yes topic is already there !! ')
    pass
else:
    new_topic   = confluent_kafka.admin.NewTopic(topic_name, 1, 1) # partitions=1, replicas=1
    kafka_admin.create_topics([new_topic,])
    # print("topic created successfully ==> ", new_topic)

yes topic is already there !! 


In [140]:
from datetime import datetime
int(datetime.now().timestamp()) * 1000

1647206744000

# Final Function to write data to kafka

In [ ]:
%%time
# pip install confluent-kafka
# pip install pandas
# pip install kafka

import confluent_kafka.admin, pprint
from kafka import KafkaProducer
import pandas as pd
from datetime import datetime
import json


def write_to_kafka(filename):
    """Function to write the data to kafka in a proper format"""
    
    try:
        with open("config.json") as jfile:
            config_data = json.load(jfile)

        rewrite_timestamp = config_data["rewrite_timestamp"]
        data = pd.read_json(filename, convert_dates=False)

        # IF REWRITE TIMESTAMP IS ENABLE THEN OVERWRITE TO CURRENT TIMESTAMP VALUE IN EPOCH
        if rewrite_timestamp.lower() == 'enable':
            data['timestamp'] = int(datetime.now().timestamp()) * 1000

        # PICKING THE TOPIC NAME & SERVER DETAILS FROM JSON FILE
        topic_name =  config_data["topic_name"]
        kafka_host = config_data["kafka_host"]
        conf = {'bootstrap.servers': kafka_host}

        # MAKING KAFKA_ADMIN OBJECT USING COLFLUENT KAFKA
        kafka_admin = confluent_kafka.admin.AdminClient(conf)

        # MAKING KAFKA PRODUCER OBJECT USING KAFKA-PYTHON
        kafkaObj = KafkaProducer(bootstrap_servers=kafka_host, api_version=(0, 10, 1))

        # LIST OUT ALL THE TOPICS -- RETURN THE RESULT IN DICTIONARY FORMAT
        topic_list = kafka_admin.list_topics().topics 
        if  topic_name in list(topic_list.keys()):
            pass
        else:
            new_topic   = confluent_kafka.admin.NewTopic(topic_name, 1, 1) # partitions=1, replicas=1
            kafka_admin.create_topics([new_topic,])

        # LOOPING THROUGH THE DATAFRAME & SENDING 1 FULL ROW AS 1 LIST AS A 1 MSG
        for idx in range(data.shape[0]):
            prepare_message = str(data.iloc[idx,:].values.tolist())
            kafkaObj.send(topic_name, value=prepare_message.encode('utf-8'))
            print(prepare_message)

        print(topic_name, kafka_host)
    
    except Exception as e:
        import traceback
        print("Error occured while writing data to kafka ===> ", str(e))
        print(traceback.format_exc())

if __name__ == '__main__':
    
    # CALLING THE FUNCTION -- PASSING THE FILENAME TO IT
    write_to_kafka(filename="1646893713217-service (output from script).json")

# Consumer code to take data from kafka

In [ ]:
from kafka import KafkaConsumer

try:
    topic_name = 'test_topic_1'
    consumer = KafkaConsumer(topic_name, bootstrap_servers = ['localhost:9092'], api_version=(0, 10), consumer_timeout_ms=1000) # set consumer for limited time with the last parameter

    for msg in consumer:
        print(msg.value)
except Exception as e:
    print("Some Error occured while reading message from consumer! ")

### Sending by each col value ... stiching col by col value under iterrows()

In [80]:
topic_name = 'test_topic_1'
for index, row in df.iterrows():
    prepare_message = ''
    for col in df.columns:
        prepare_message += str(row[col])+ ', '
    print("Final Message is ==>\n" ,prepare_message)
    
    break

Final Message is ==>
 2022-03-10-06:28:33, Client, icmp, 10.236.141.200, 04b0e711fc18, nan, 183274952, nan, nan, nan, nan, 0, 25, 0, 25, 25, 0, 2186, 0, 2186, 2186, 


In [73]:
topic_name = 'test_topic_1'
for index, row in df.iterrows():
    prepare_message = ''
    for col in df.columns:
        prepare_message += str(row[col])+ ', '
    print(prepare_message)
    
    break

1646893713217, Client, icmp, 10.236.141.200, 04b0e711fc18, nan, 183274952, nan, nan, nan, nan, 0, 25, 0, 25, 25, 0, 2186, 0, 2186, 2186, 


In [37]:
str(row['timestamp'])

'2022-03-10 06:28:33.217000'

In [16]:
for col in df.columns:
    print(row[col])
    print('--'*15)

2022-03-10 06:28:33.217000
------------------------------
Client
------------------------------
icmp
------------------------------
10.236.141.200
------------------------------
04b0e711fc18
------------------------------
nan
------------------------------
183274952
------------------------------
nan
------------------------------
nan
------------------------------
nan
------------------------------
nan
------------------------------
0
------------------------------
25
------------------------------
0
------------------------------
25
------------------------------
25
------------------------------
0
------------------------------
2186
------------------------------
0
------------------------------
2186
------------------------------
2186
------------------------------
